絵文字をマージするかどうか

In [1]:
############################################################################################################
# パラメータ設定
# 絵文字をマージするかどうか
emoji_merge = True
# マージ元
from_emoji = ["😂", "💕", "☺️", "😃", "😆", "😁", "🥲", "👍", "✨", "☀️", "😅", "🥺"]
# マージ先
to_emoji = ["🤣", "🥰", "😊", "😊", "😊", "😊", "😭", "😊", "😊", "😊", "🤣", "😭"]
############################################################################################################

ファインチューニング・テスト用のデータセットの作成

In [2]:
import json
import random
import pandas as pd


# テストの時のテストデータの取得
def get_test_data(dataset, num_per_label, completely_random, num_of_completely_random, create_json=False):

    # completely random データの取り出し
    if completely_random:
        pass
    # 各ラベルにつき、何個のデータを取り出す
    else:
        print(f"データの抽出方法：各ラベルごとにランダムに {num_per_label} 件のデータを抽出する")
        # ラベルの取得
        emojis = []
        for d in dataset:
            if d["label"] not in emojis:
                emojis.append(d["label"])
        #print(len(emojis))
        #print(emojis)

        # テストデータセットの作成
        test_dataset = []
        for i in range(0, len(emojis), 1):
            test_dataset_temp = [d for d in dataset if d["label"] == emojis[i]]
            test_dataset_temp = random.sample(test_dataset_temp, num_per_label)
            for d in test_dataset_temp:
                test_dataset.append(d)
        #print(len(test_dataset))
        #print(test_dataset)
        print(f"データは全部で {num_per_label}（各ラベルごとのデータ数） * {len(emojis)}（ラベル数） = {len(test_dataset)} 件")

        # json ファイルの作成
        if create_json:
            json_path = f"./dataset/test/test_{len(test_dataset)}_dataset.json"
            with open(json_path, 'w', encoding='utf-8') as f:
                json.dump(test_dataset, f, indent=4, ensure_ascii=False)

        return test_dataset


def get_ft_dataset(num_per_label, ft_random=True):
    ############################################################################################################
    # パラメータ設定
    # json ファイルパス
    json_path = "./dataset/top20_one_emoji_dataset.json"
    # 既存の訓練データセットのパス
    training_path = "./dataset/train_gpt/R8.json"
    ############################################################################################################
    
    # json ファイルの読み込み
    with open(json_path, 'r') as f:
        dataset = json.load(f)
    #print(len(dataset))

    # データをランダムに取り出す、テストデータの作成と一緒
    if ft_random:
        ft_dataset = get_test_data(dataset=dataset,
                                   num_per_label=num_per_label,
                                   completely_random=False,
                                   num_of_completely_random=False
                                   )
        #print(len(ft_dataset))
        #print(ft_dataset[0])

        ft_dataset = [
            {"messages": [{"role": "user", "content": f"{d['text']}というツイートに最も相応しい絵文字はどれ？\n選択肢：😅、☀️、☺️、😆、😂、😭、✨、🤣、🥺、🎉、🥰、🥲、😃、💕、💦、😇、😊、👍、😁、🤔"},
                          {"role": "assistant", "content": d["label"]}]}
            for d in ft_dataset
        ]
        #print(ft_dataset)

        ft_dataset = pd.DataFrame(ft_dataset)
        ft_dataset.to_json("./dataset/train_gpt/dataset_temp.jsonl", force_ascii=False, lines=True, orient="records")
    # 既存のデータセットを取得する
    else:
        # 既存のデータセットを読み込む
        with open(training_path, 'r') as f:
            ft_dataset = [json.loads(d) for d in f.readlines()]
        #print(len(ft_dataset))
        #print(ft_dataset[0])

        ft_dataset = [
            {"messages": [
                {"role": "user", "content": d['messages'][0]['content']},
                {"role": "assistant", "content": d['messages'][1]['content']}
            ]}
            for d in ft_dataset
        ]
        #print(len(ft_dataset))
        #print(ft_dataset[0])

        ft_dataset = pd.DataFrame(ft_dataset)
        ft_dataset.to_json("./dataset/train_gpt/dataset_temp.jsonl", force_ascii=False, lines=True, orient="records")

推論

In [ ]:
import os
import json
import time
import emoji
import openai
from openai import OpenAI


def main():

    ############################################################################################################
    # パラメータ設定
    # id
    result_id = "GPT35_1"
    # json ファイルのパス
    json_path = "./dataset/top20_one_emoji_dataset.json"
    # テストデータセットを作成するかどうか
    create_test_dataset = False
    # 既存のテストデータを読み込む用
    test_path = "./dataset/test/T8h.json"
    # 使うモデル
    model = "gpt-3.5-turbo-1106"
    # temperature（出力のランダム性）
    temperature = 0
    # テストだけの時のテストは completely random かどうか、False 各ラベルにつき、何個のデータを取り出すか
    completely_random = False
    # completely random の時の取り出すデータの数
    num_of_completely_random = 200
    # ファインチューニング・テストだけの時の各ラベルにつき、何個のデータを取り出すか
    num_per_label = 20
    # json ファイルに出力かどうか
    create_json = True
    ############################################################################################################

    # api key をロードする
    client = OpenAI()

    # json ファイルの読み込み
    with open(json_path, 'r') as f:
        topn_one_emoji_dataset = json.load(f)
    #print(len(topn_one_emoji_dataset))
    #print(topn_one_emoji_dataset[2])

    # テスト
    if create_test_dataset:
        test_dataset = get_test_data(
            dataset=topn_one_emoji_dataset,
            num_per_label=num_per_label,
            completely_random=completely_random,
            num_of_completely_random=num_of_completely_random,
            create_json=create_json
            )
    else:
        with open(test_path, 'r') as f:
            test_dataset = json.load(f)

    # 絵文字をマージする
    if emoji_merge:
        test_dataset_temp = []
        for d in test_dataset:
            if d["label"] in from_emoji:
                d["label"] = to_emoji[from_emoji.index(d["label"])]
                test_dataset_temp.append(d)
            else:
                test_dataset_temp.append(d)
        test_dataset = test_dataset_temp

    output = []
    correct_output = []
    wrong_output = []
    for d in test_dataset:

        # GPT の入力形式に変換する
        messages = [
            {"role": "user", "content": f"{d['text']}というツイートに最も相応しい絵文字を選択肢の中から一つ選んでください。\
なお、選んだ一つの絵文字だけ回答してください。\
\nただし、🎉は「おめでとう」のようなお祝いの言葉の後にしか付けない、\
🥰は「好き」、「幸せ」、「素敵」を表現する、\
😇は「終わった」、「もうダメ」、「天使」に関連して使う、\
💦は「ストレス」、「緊張」、「汗」、「暑さ」に関連して使う\
という条件がある。\
\n選択肢：😊、🤣、😭、🎉、🥰、😇、💦、🤔"}
        ]

        # レスポンスを生成する
        try:
            time.sleep(5)
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=temperature
                )
        except openai.APITimeoutError:
            time.sleep(120)
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=temperature
                )

        # モデルが生成した絵文字は一つだけかどうかをチェック
        # wrong
        if emoji.emoji_count(response.choices[0].message.content, unique=True) != 1:
            output_temp = {
                "モデル": response.model,
                "temperature": temperature,
                "入力": {"role": messages[0]['role'], "content": messages[0]['content']},
                "出力": {"role": response.choices[0].message.role, "content": response.choices[0].message.content},
                "プロンプトトークン数": response.usage.prompt_tokens,
                "出力トークン数": response.usage.completion_tokens,
                "総トークン数": response.usage.total_tokens,
                "予測": emoji.distinct_emoji_list(response.choices[0].message.content),
                "正解": d["label"],
                "正誤": "false"
                }
            output.append(output_temp)
            wrong_output.append(output_temp)
        # correct
        else:
            # 正解
            if emoji.distinct_emoji_list(response.choices[0].message.content)[0] == d["label"]:
                output_temp = {
                    "モデル": response.model,
                    "temperature": temperature,
                    "入力": {"role": messages[0]['role'], "content": messages[0]['content']},
                    "出力": {"role": response.choices[0].message.role, "content": response.choices[0].message.content},
                    "プロンプトトークン数": response.usage.prompt_tokens,
                    "出力トークン数": response.usage.completion_tokens,
                    "総トークン数": response.usage.total_tokens,
                    "予測": emoji.distinct_emoji_list(response.choices[0].message.content)[0],
                    "正解": d["label"],
                    "正誤": "true"
                    }
                output.append(output_temp)
                correct_output.append(output_temp)
            # 不正解
            else:
                output_temp = {
                    "モデル": response.model,
                    "temperature": temperature,
                    "入力": {"role": messages[0]['role'], "content": messages[0]['content']},
                    "出力": {"role": response.choices[0].message.role, "content": response.choices[0].message.content},
                    "プロンプトトークン数": response.usage.prompt_tokens,
                    "出力トークン数": response.usage.completion_tokens,
                    "総トークン数": response.usage.total_tokens,
                    "予測": emoji.distinct_emoji_list(response.choices[0].message.content)[0],
                    "正解": d["label"],
                    "正誤": "false"
                    }
                output.append(output_temp)
                correct_output.append(output_temp)
    print(f"絵文字が一つだけの出力の数： {len(correct_output)}")
    print(f"絵文字が一つじゃないの出力の数： {len(wrong_output)}")

    # json ファイルの出力
    # 全部
    json_path = f"./results/{result_id}/{model.replace('personal::', '').replace('-', '_')}_output.json"
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(output, f, indent=4, ensure_ascii=False)

    # 絵文字が一つだけの出力
    json_path = f"./results/{result_id}/{model.replace('personal::', '').replace('-', '_')}_correct_output.json"
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(correct_output, f, indent=4, ensure_ascii=False)

    # 絵文字が一つじゃない出力
    json_path = f"./results/{result_id}/{model.replace('personal::', '').replace('-', '_')}_wrong_output.json"
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(wrong_output, f, indent=4, ensure_ascii=False)
    

if __name__ == "__main__":
    main()

ラベルの取得・マッピング

In [4]:
import json


# ラベルの取得
def get_label(json_path):
    with open(json_path, 'r') as f:
        dataset = json.load(f)

    # ラベルの取得（出現回数に関係ない）
    label = []
    for d in dataset:
        if d["正解"] not in label:
            label.append(d["正解"])
   
    return label

推論、精度計算

In [ ]:
import json
import numpy as np
from sklearn.metrics import classification_report, ConfusionMatrixDisplay


def main():

    ############################################################################################################
    # パラメータ設定
    # id
    result_id = "GPT35_1"
    # モデル
    model = "gpt-3.5-turbo-1106"
    # json ファイルパス
    json_path = "./results/GPT35_1/gpt_3.5_turbo_1106_output.json"
    ############################################################################################################

    # ラベルの取得
    label = get_label(json_path=json_path)
    #print(len(label))
    print('、'.join(label))
    
    # json ファイルの読み込み
    json_path = f"./results/{result_id}/{model.replace('personal::', '').replace('-', '_')}_output.json"
    with open(json_path, 'r') as f:
        output = json.load(f)
    #print(len(output))
    json_path = f"/results/{result_id}/{model.replace('personal::', '').replace('-', '_')}_correct_output.json"
    with open(json_path, 'r') as f:
        correct_output = json.load(f)
    #print(len(correct_output))

    # 用意したラベル以外の絵文字が予測された数
    num_of_not_in_label = 0
    for d in correct_output:
        if d["予測"] not in label:
            num_of_not_in_label = num_of_not_in_label + 1
    print(f"用意したラベル以外の絵文字が予測された数: {num_of_not_in_label}")

    # Acc 計算
    num_of_true = len([d["正誤"] for d in output if d["正誤"] == "true"])
    print(f"正解数： {num_of_true}")
    print(f"Acc： {(num_of_true / len(output)) * 100}")

    # P R F1 計算
    label_gold = np.array([d["正解"] for d in correct_output])
    label_pred = np.array([d["予測"] for d in correct_output])
    print(classification_report(y_true=label_gold, y_pred=label_pred, labels=label, zero_division=0.0))

    ConfusionMatrixDisplay.from_predictions(y_true=label_gold, y_pred=label_pred, labels=label)


if __name__ == "__main__":
    main()